In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle

import os
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

In [2]:
import pandas as pd
import os
import pickle
import time
import json
from collections import defaultdict
from openai import OpenAI
from datetime import datetime


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")


In [3]:
import subprocess
#https://www.youtube.com/watch?v=dCHh2HacOYk
# Trump MELTDOWN on Walmart + Saudi news
# 0UtHedAXIkA
# Indians Finally Admit Pakistan Won

# https://www.youtube.com/watch?v=xJ_omX4Ejqw
# India Beat Pakistan's Air Defense

# https://www.youtube.com/watch?v=7tqYgfhCwb4&t=154s&pp=ygUOc21pbGUgMiBqYW5uYWg%3D
# Indians Declare WAR on Aamir Khan



video_id = "7tqYgfhCwb4"
video_url = f"https://www.youtube.com/watch?v={video_id}"


command = [
    "yt-dlp",
    "--skip-download",
    "--write-comments",
    "--no-warnings",
    "--output", f"comments_scraped/{video_id}_{timestamp}",
    video_url
]

try:
    result = subprocess.run(command, check=True, text=True, capture_output=True)
    print("yt-dlp ran successfully.")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("yt-dlp failed.")
    print(e.stderr)

yt-dlp ran successfully.
[youtube] Extracting URL: https://www.youtube.com/watch?v=7tqYgfhCwb4
[youtube] 7tqYgfhCwb4: Downloading webpage
[youtube] 7tqYgfhCwb4: Downloading tv client config
[youtube] 7tqYgfhCwb4: Downloading tv player API JSON
[youtube] 7tqYgfhCwb4: Downloading ios player API JSON
[youtube] 7tqYgfhCwb4: Downloading m3u8 information
[youtube] Downloading comment section API JSON
[youtube] Downloading ~1454 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~1454)
[youtube]     Downloading comment API JSON reply thread 1 (1/~1454)
[youtube] Downloading comment API JSON page 2 (29/~1454)
[youtube] Downloading comment API JSON page 3 (49/~1454)
[youtube] Downloading comment API JSON page 4 (69/~1454)
[youtube] Downloading comment API JSON page 5 (89/~1454)
[youtube] Downloading comment API JSON page 6 (109/~1454)
[youtube]     Downloading comment API JSON reply thread 1 (122/~1454)
[youtube]     Downloading comment API JSON

In [4]:



# Load your JSON file
with open(f"comments_scraped/{video_id}_{timestamp}.info.json", "r", encoding="utf-8") as f:
    all_data=json.load(f)        
    raw_comments = all_data['comments']
    title=all_data['title']
    uploader_id=all_data['uploader_id']



In [5]:
title,uploader_id

('Indians Declare WAR on Aamir Khan', '@smiletojannah')

In [6]:
# Group comments by ID and organize replies under their parent
comment_dict = {}
replies_grouped = defaultdict(list)

# First pass to separate top-level and replies
for comment in raw_comments:
    print(comment)
    if comment.get("parent") and comment["parent"] != "root":
        replies_grouped[comment["parent"]].append({
            "id": comment["id"],
            "text": comment["text"],
            "author": comment["author"],
            "likes": comment["like_count"],
            "timestamp": comment["timestamp"]
        })
    else:
        comment_dict[comment["id"]] = {
            "id": comment["id"],
            "text": comment["text"],
            "author": comment["author"],
            "likes": comment["like_count"],
            "timestamp": comment["timestamp"],
            "replies": []
        }

# Second pass to attach replies to top-level comments
for parent_id, replies in replies_grouped.items():
    if parent_id in comment_dict:
        comment_dict[parent_id]["replies"] = replies

# Convert dict to list
structured_comments = list(comment_dict.values())

# Print a few samples
for comment in structured_comments[:3]:  # limit to 3 for brevity
    print(f"Comment by {comment['author']}: {comment['text']}")
    for reply in comment["replies"]:
        print(f"  ↳ Reply by {reply['author']}: {reply['text']}")
    print("-" * 60)

{'id': 'UgzppYSjDCq3I3-zEzp4AaABAg', 'parent': 'root', 'text': 'Do you Qurbani in Kenya this year (with a cow share costing £48): http://spot-project.givebrite.com/s2j-qurbani-2025', 'like_count': 122, 'author_id': 'UCeccrNGLzk6d0M_vaomD3LQ', 'author': '@smiletojannah', 'author_thumbnail': 'https://yt3.ggpht.com/ytc/AIdro_kmM45yxhtZFVg8rvCXicdd1Lc_BLPVKpolFj_-Bp7DpR4=s88-c-k-c0x00ffffff-no-rj', 'author_is_uploader': True, 'author_is_verified': True, 'author_url': 'https://www.youtube.com/@smiletojannah', 'is_favorited': False, '_time_text': '1 day ago', 'timestamp': 1747526400, 'is_pinned': True}
{'id': 'UgzppYSjDCq3I3-zEzp4AaABAg.AIEgchpKzXsAIEgtVC5hPL', 'parent': 'UgzppYSjDCq3I3-zEzp4AaABAg', 'text': 'Is there any link for our brothers in Ga,,za', 'like_count': 4, 'author_id': 'UCDYaGZoJDyRKXDf-h570Zzg', 'author': '@Yassinovish007', 'author_thumbnail': 'https://yt3.ggpht.com/39-q3TdmXo-vckHfSoSxNhiPs1bsFEK131xDymV0FsayPhYdfv0AUxrsxl-92fDGsA3bnoB7yQ=s88-c-k-c0x00ffffff-no-rj', 'author

In [7]:
base_filename=f'{video_id}'
rows = []
for comment in structured_comments:
    rows.append({
        "id": comment["id"],
        "text": comment["text"],
        "author": comment["author"],
        "likes": comment["likes"],
        "timestamp": comment["timestamp"],
        "is_reply": False,
        "parent_id": None
    })
    for reply in comment.get("replies", []):
        rows.append({
            "id": reply["id"],
            "text": reply["text"],
            "author": reply["author"],
            "likes": reply["likes"],
            "timestamp": reply["timestamp"],
            "is_reply": True,
            "parent_id": comment["id"]
        })

# Create a DataFrame
df = pd.DataFrame(rows)



In [8]:
df

,id,text,author,likes,timestamp,is_reply,parent_id
0,UgzppYSjDCq3I3-zEzp4AaABAg,Do you Qurbani in Kenya this year (with a cow ...,@smiletojannah,122,1747526400,False,None
1,UgzppYSjDCq3I3-zEzp4AaABAg.AIEgchpKzXsAIEgtVC5hPL,"Is there any link for our brothers in Ga,,za",@Yassinovish007,4,1747526400,True,UgzppYSjDCq3I3-zEzp4AaABAg
2,UgzppYSjDCq3I3-zEzp4AaABAg.AIEgchpKzXsAIEiFcK9DR2,In Sha Allah,@muhammadjalal2335,0,1747526400,True,UgzppYSjDCq3I3-zEzp4AaABAg
3,UgzppYSjDCq3I3-zEzp4AaABAg.AIEgchpKzXsAIEjhhqE-FR,Walaikumsalam,@M.kingfaze777,0,1747526400,True,UgzppYSjDCq3I3-zEzp4AaABAg
4,UgzppYSjDCq3I3-zEzp4AaABAg.AIEgchpKzXsAIElbqw5mBv,Brother im very upset with you \n4:23 4:25 4...,@mkodyglobalsouthsoldier,0,1747526400,True,UgzppYSjDCq3I3-zEzp4AaABAg
...,...,...,...,...,...,...,...
1449,UgwzIWrX8G8-s1j1SCR4AaABAg.AIEgeriR_EyAIEgi8BgOWC,Yes my bro. I hope you're well,@smiletojannah,3,1747526400,True,UgwzIWrX8G8-s1j1SCR4AaABAg
1450,UgwzIWrX8G8-s1j1SCR4AaABAg.AIEgeriR_EyAIEhh2W--h3,@smiletojannah assalamu alaykum. But he isn't-,@زكريآ-مسلم,0,1747526400,True,UgwzIWrX8G8-s1j1SCR4AaABAg
1451,UgwzIWrX8G8-s1j1SCR4AaABAg.AIEgeriR_EyAIEivtsUbcM,@زكريآ-مسلمyes you’re right,@smiletojannah,0,1747526400,True,UgwzIWrX8G8-s1j1SCR4AaABAg
1452,UgwzIWrX8G8-s1j1SCR4AaABAg.AIEgeriR_EyAIEkTDqdAzK,I find thank you for the reply are you real pe...,@mehedimasud5195,0,1747526400,True,UgwzIWrX8G8-s1j1SCR4AaABAg


In [9]:
df.sort_values(by=['likes'],inplace=True,ascending=False)

In [10]:
# Export to CSV
df.to_csv(f"{base_filename}.csv", index=False)

# Export to Pickle
with open(f"{base_filename}.pkl", "wb") as f:
    pickle.dump(structured_comments, f)

print("Export completed: CSV and PKL saved.")


Export completed: CSV and PKL saved.


In [11]:
df_comment1=df[df['is_reply']==False]

In [12]:
df_comment1

,id,text,author,likes,timestamp,is_reply,parent_id
1375,UgzvFIWRDe-fNtZY2AF4AaABAg,Majority of Bollywood stayed quiet but they wi...,@Fkb123,1200,1747526400,False,None
857,Ugz00Tn7VAgn1Imy6NB4AaABAg,"Hello from Türkiye, Free Palestina Free Kashmi...",@ege5193,922,1747526400,False,None
966,UgwnwWUKuWMbHu37wMt4AaABAg,"So they lost against Pakistan, now they take t...",@abayomrunkojemusa9086,828,1747526400,False,None
722,UgwIuGTzdHLxr7aoIu14AaABAg,Poor india. 😂 Support Israel but israel never ...,@azmirazri2761,813,1747526400,False,None
1298,Ugw2WYFhXw8KOKzg-sd4AaABAg,"The Prophet (ﷺ) said: ""There are two blessings...",@HamzaGW,483,1747526400,False,None
...,...,...,...,...,...,...,...
512,Ugw-LRNwHqK0jFZGshJ4AaABAg,Let's see you speak against the Pakistan gover...,@ashinanto,0,1747526400,False,None
511,UgzPvj9voBZHND09mId4AaABAg,P4Jeets going Crazy,@shawnkay5462,0,1747526400,False,None
510,UgwT8_xoug0gLSacB7B4AaABAg,They are Jealous of successfull Muslim actors,@faisaliqbalian,0,1747526400,False,None
508,UgzYtTOSevOEwHz_Uel4AaABAg,"How could India come after Aamir Khan, when hi...",@Asenueh,0,1747526400,False,None


In [13]:
comment1_list=list(df_comment1['text'])

In [14]:
print(comment1_list[:2])

['Majority of Bollywood stayed quiet but they will only go after the Muslims because pajeet hate the fact khans been running Bollywood for decades 😂', 'Hello from Türkiye, Free Palestina Free Kashmir, Free Muslim.']


In [15]:
k=5
topkcomments_list=comment1_list[:int(0.05*len(comment1_list))]

In [16]:
len(topkcomments_list),len(comment1_list)

(37, 748)

In [17]:
topkcomments_str='\n'.join(topkcomments_list)

In [18]:
api_key='sk-bedceae2ceba437f944db22706354095'

In [19]:
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com",
)

system_prompt = """
The user will provide a list of YouTube comments. Please analyze all the comments together and generate a single, structured JSON object summarizing the overall qualitative dynamics.

EXAMPLE INPUT:
["Great job! This video really opened my eyes.", "What a biased take. Shameful.", "😂😂 you're so clueless it's funny.", "Pakistan zindabad!", "Link to giveaway 👉 http://spamlink"]

EXAMPLE JSON OUTPUT:
{
  "overall_sentiment_distribution": {
    "positive": 1,
    "neutral": 1,
    "negative": 3
  },
  "dominant_emotions": ["anger", "sarcasm", "joy"],
  "toxic_comment_count": 2,
  "controversy_score": 0.75,
  "key_topics": ["bias in media", "nationalism", "truth and misinformation"],
  "frequent_bias_or_group_mentions": ["Pakistan", "India", "YouTube creators"],
  "sarcasm_detected": true,
  "languages_detected": ["English", "Urdu"],
  "spam_comment_count": 1,
  "summary": "The comment section is emotionally charged with a mix of national pride, strong criticism, and sarcasm. There's significant polarization, and a moderate amount of toxicity and spam. These patterns suggest the topic is controversial and has drawn viewers from diverse backgrounds with opposing views."
}

Please return the analysis in this format. Focus on producing insightful, unbiased feedback that helps a content creator understand audience sentiment, engagement, controversy, and risks.
"""

user_prompt = topkcomments_str

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    response_format={
        'type': 'json_object'
    }
)

print(json.loads(response.choices[0].message.content))

{'overall_sentiment_distribution': {'positive': 2, 'neutral': 1, 'negative': 21}, 'dominant_emotions': ['anger', 'sarcasm', 'pride', 'disgust'], 'toxic_comment_count': 15, 'controversy_score': 0.9, 'key_topics': ['religious conflict', 'nationalism', 'Bollywood', 'India-Pakistan relations', 'Muslim identity'], 'frequent_bias_or_group_mentions': ['Muslims', 'Hindus', 'Pakistan', 'India', 'Bollywood actors'], 'sarcasm_detected': True, 'languages_detected': ['English', 'Urdu'], 'spam_comment_count': 1, 'summary': "The comment section is highly polarized, with a strong negative sentiment predominantly driven by discussions on religious and national identity, particularly focusing on the Muslim community in India and the India-Pakistan conflict. There's a significant amount of toxicity, with many comments expressing anger, sarcasm, and disgust towards opposing views. The controversy score is high, indicating a deeply divided audience. Key topics revolve around religious conflict, nationalism

In [20]:
llm_analysis=json.loads(response.choices[0].message.content)
llm_analysis['title']=title
llm_analysis['uploader_id']=uploader_id



In [21]:

# Construct the filename
filename = f"insights/{video_id}_insight1_{timestamp}.json"
# Save to JSON file
with open(filename, "w", encoding="utf-8") as f:
    json.dump(llm_analysis, f, ensure_ascii=False, indent=2)
print(f"Saved analysis to {filename}")    

Saved analysis to insights/7tqYgfhCwb4_insight1_20250519_031936.json
